# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [2]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=0&sort=popularity'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [3]:
# headers definition
headers = {'User-Agent': 'Mozilla/5.0 (Windows Phone 10.0; Android 6.0.1; Microsoft; RM-1152)'}


In [4]:
# Your code
response = requests.get(url, headers=headers).json()
response


{'total_count': 25283,
 'pagination': {'page_count': 301, 'current_page': 1, 'per_page': 84},
 'sort': 'popularity',
 'articles': [{'sku': 'PU116D04U-Q11',
   'name': 'SMASH - Baskets basses - black/white',
   'price': {'original': '38,95\xa0€',
    'promotional': '35,05\xa0€',
    'has_different_prices': True,
    'has_different_original_prices': False,
    'has_different_promotional_prices': True,
    'has_discount_on_selected_sizes_only': False},
   'url_key': 'puma-smash-baskets-basses-blackwhite-pu116d04u-q11',
   'media': [{'path': 'PU/11/6D/04/UQ/11/PU116D04U-Q11@3.jpg',
     'role': 'DEFAULT',
     'packet_shot': False}],
   'brand_name': 'Puma',
   'is_premium': False,
   'flags': [{'key': 'discountRate',
     'value': 'Jusqu’à -10%',
     'tracking_value': 'discount rate'},
    {'key': 'sponsored',
     'value': 'Sponsorisé',
     'tracking_value': 'sponsored'}],
   'tracking_information': {'metrigo_impression_urls': ['https://ccp-et.adtechlab.zalan.do/event/sbv?z=742f1b6f-61

#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [5]:
# Get the total number of pages
num_pages = response['pagination']['page_count']
print(num_pages)
# Your code
articles = pd.DataFrame(response['articles'])
articles

301


,sku,name,price,url_key,media,brand_name,is_premium,flags,tracking_information,product_group,delivery_promises,amount
0,PU116D04U-Q11,SMASH - Baskets basses - black/white,"{'original': '38,95 €', 'promotional': '35,05 ...",puma-smash-baskets-basses-blackwhite-pu116d04u...,[{'path': 'PU/11/6D/04/UQ/11/PU116D04U-Q11@3.j...,Puma,False,"[{'key': 'discountRate', 'value': 'Jusqu’à -10...",{'metrigo_impression_urls': ['https://ccp-et.a...,shoe,[],NaN
1,TI114D00Q-Q11,BROOKLYN HIKER - Baskets montantes - black,"{'original': '99,90 €', 'promotional': '88,95 ...",timberland-brooklyn-hiker-baskets-montantes-bl...,[{'path': 'TI/11/4D/00/QQ/11/TI114D00Q-Q11@9.j...,Timberland,False,"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",{'metrigo_impression_urls': ['https://ccp-et.a...,shoe,[],NaN
2,WO813D00O-K11,SANDRA - Baskets basses - navy,"{'original': '69,95 €', 'promotional': '38,45 ...",woden-sandra-pearl-baskets-basses-wo813d00o-k11,[{'path': 'WO/81/3D/00/OK/11/WO813D00O-K11@3.j...,Woden,False,"[{'key': 'discountRate', 'value': 'Jusqu’à -45...",{'metrigo_impression_urls': ['https://ccp-et.a...,shoe,[],NaN
3,NI114D0C1-Q12,AIR MAX TAILWIND IV - Baskets basses - black,"{'original': '139,95 €', 'promotional': '41,95...",nike-sportswear-air-max-tailwind-baskets-basse...,[{'path': 'NI/11/4D/0C/1Q/12/NI114D0C1-Q12@10....,Nike Sportswear,False,"[{'key': 'discountRate', 'value': 'Jusqu’à -70...",NaN,shoe,[],NaN
4,AD116D008-A11,STAN SMITH - Baskets basses - white/green,"{'original': '64,95 €', 'promotional': '45,45 ...",adidas-originals-stan-smith-baskets-basses-bla...,[{'path': 'AD/11/6D/00/8A/11/AD116D008-A11@18....,adidas Originals,False,"[{'key': 'discountRate', 'value': '-30%', 'tra...",NaN,shoe,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
79,STF24G06N-M11,TEENAGER - T-shirt à manches longues - bottle ...,"{'original': '15,95 €', 'promotional': '9,60 €...",staccato-teenager-t-shirt-a-manches-longues-bo...,[{'path': 'ST/F2/4G/06/NM/11/STF24G06N-M11@9.1...,Staccato,False,"[{'key': 'discountRate', 'value': '-40%', 'tra...",NaN,clothing,[],NaN
80,NI113D09D-A11,AIR MAX 270 REACT - Baskets basses - summit wh...,"{'original': '134,95 €', 'promotional': '53,95...",nike-sportswear-air-max-270-react-baskets-bass...,[{'path': 'NI/11/3D/09/DA/11/NI113D09D-A11@3.j...,Nike Sportswear,False,"[{'key': 'discountRate', 'value': 'Jusqu’à -60...",NaN,shoe,[],NaN
81,C2723L015-B11,Veste en jean - beige,"{'original': '79,00 €', 'promotional': '31,60 ...",catimini-veste-en-jean-beige-c2723l015-b11,[{'path': 'C2/72/3L/01/5B/11/C2723L015-B11@11....,Catimini,False,"[{'key': 'discountRate', 'value': '-60%', 'tra...",NaN,clothing,[],NaN
82,NI126D007-Q11,SUIT CORE - veste en sweat zippée - black/white,"{'original': '54,95 €', 'promotional': '49,45 ...",nike-sportswear-suit-core-set-pantalon-de-surv...,[{'path': 'NI/12/6D/00/7Q/11/NI126D007-Q11@12....,Nike Sportswear,False,"[{'key': 'discountRate', 'value': '-10%', 'tra...",NaN,clothing,[],NaN


In [10]:
def extract_discount(x):
    for l in x[0]:
        if x[0][l] == 'discountRate':
            print(x[0]['value'])
            return x[0]['value'][x[0]['value'].find('-'):x[0]['value'].find('%')]
    return 0


In [11]:
articles['discount'] = articles['flags'].apply(extract_discount)
articles

Jusqu’à -10%
Jusqu’à -45%
Jusqu’à -70%
-30%
-50%
-15%
-30%
-30%
-35%
Jusqu’à -40%
-65%
Jusqu’à -45%
-50%
-10%
-40%
-25%
-20%
Jusqu’à -55%
-45%
Jusqu’à -10%
-20%
-45%
-60%
-15%
-10%
-10%
-70%
-55%
-70%
-40%
-20%
-10%
Jusqu’à -30%
Jusqu’à -45%
-40%
Jusqu’à -20%
-45%
Jusqu’à -70%
-20%
-50%
-35%
-20%
-60%
-40%
-50%
-10%
-70%
-45%
-25%
-30%
Jusqu’à -10%
-50%
-65%
-70%
-10%
-30%
-10%
-70%
-70%
-35%
-60%
-25%
-60%
-45%
-70%
-65%
-25%
-20%
-30%
-20%
-70%
Jusqu’à -55%
-40%
-50%
-60%
-30%
-20%
-25%
-40%
Jusqu’à -60%
-60%
-10%
-25%


,sku,name,price,url_key,media,brand_name,is_premium,flags,tracking_information,product_group,delivery_promises,amount,discount
0,PU116D04U-Q11,SMASH - Baskets basses - black/white,"{'original': '38,95 €', 'promotional': '35,05 ...",puma-smash-baskets-basses-blackwhite-pu116d04u...,[{'path': 'PU/11/6D/04/UQ/11/PU116D04U-Q11@3.j...,Puma,False,"[{'key': 'discountRate', 'value': 'Jusqu’à -10...",{'metrigo_impression_urls': ['https://ccp-et.a...,shoe,[],NaN,-10
1,TI114D00Q-Q11,BROOKLYN HIKER - Baskets montantes - black,"{'original': '99,90 €', 'promotional': '88,95 ...",timberland-brooklyn-hiker-baskets-montantes-bl...,[{'path': 'TI/11/4D/00/QQ/11/TI114D00Q-Q11@9.j...,Timberland,False,"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",{'metrigo_impression_urls': ['https://ccp-et.a...,shoe,[],NaN,0
2,WO813D00O-K11,SANDRA - Baskets basses - navy,"{'original': '69,95 €', 'promotional': '38,45 ...",woden-sandra-pearl-baskets-basses-wo813d00o-k11,[{'path': 'WO/81/3D/00/OK/11/WO813D00O-K11@3.j...,Woden,False,"[{'key': 'discountRate', 'value': 'Jusqu’à -45...",{'metrigo_impression_urls': ['https://ccp-et.a...,shoe,[],NaN,-45
3,NI114D0C1-Q12,AIR MAX TAILWIND IV - Baskets basses - black,"{'original': '139,95 €', 'promotional': '41,95...",nike-sportswear-air-max-tailwind-baskets-basse...,[{'path': 'NI/11/4D/0C/1Q/12/NI114D0C1-Q12@10....,Nike Sportswear,False,"[{'key': 'discountRate', 'value': 'Jusqu’à -70...",NaN,shoe,[],NaN,-70
4,AD116D008-A11,STAN SMITH - Baskets basses - white/green,"{'original': '64,95 €', 'promotional': '45,45 ...",adidas-originals-stan-smith-baskets-basses-bla...,[{'path': 'AD/11/6D/00/8A/11/AD116D008-A11@18....,adidas Originals,False,"[{'key': 'discountRate', 'value': '-30%', 'tra...",NaN,shoe,[],NaN,-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,STF24G06N-M11,TEENAGER - T-shirt à manches longues - bottle ...,"{'original': '15,95 €', 'promotional': '9,60 €...",staccato-teenager-t-shirt-a-manches-longues-bo...,[{'path': 'ST/F2/4G/06/NM/11/STF24G06N-M11@9.1...,Staccato,False,"[{'key': 'discountRate', 'value': '-40%', 'tra...",NaN,clothing,[],NaN,-40
80,NI113D09D-A11,AIR MAX 270 REACT - Baskets basses - summit wh...,"{'original': '134,95 €', 'promotional': '53,95...",nike-sportswear-air-max-270-react-baskets-bass...,[{'path': 'NI/11/3D/09/DA/11/NI113D09D-A11@3.j...,Nike Sportswear,False,"[{'key': 'discountRate', 'value': 'Jusqu’à -60...",NaN,shoe,[],NaN,-60
81,C2723L015-B11,Veste en jean - beige,"{'original': '79,00 €', 'promotional': '31,60 ...",catimini-veste-en-jean-beige-c2723l015-b11,[{'path': 'C2/72/3L/01/5B/11/C2723L015-B11@11....,Catimini,False,"[{'key': 'discountRate', 'value': '-60%', 'tra...",NaN,clothing,[],NaN,-60
82,NI126D007-Q11,SUIT CORE - veste en sweat zippée - black/white,"{'original': '54,95 €', 'promotional': '49,45 ...",nike-sportswear-suit-core-set-pantalon-de-surv...,[{'path': 'NI/12/6D/00/7Q/11/NI126D007-Q11@12....,Nike Sportswear,False,"[{'key': 'discountRate', 'value': '-10%', 'tra...",NaN,clothing,[],NaN,-10


#### Display the trending brand in DataFrame

In [12]:
# your code
print(articles.brand_name.value_counts().nlargest(1))

Nike Sportswear    14
Name: brand_name, dtype: int64


#### Display the brand with maximal total discount (sum of discounts on all goods)

In [23]:
#Our data is still text. Convert prices into numbers.
# your code
articles["discount"] = articles["discount"].astype(str).astype(int)
articles.groupby('brand_name')['discount'].sum().nsmallest(1)

brand_name
Nike Sportswear   -580
Name: discount, dtype: int32

#### Display the brands without discount at all

In [32]:
# your code
articles[articles.discount == 0]['brand_name'].unique()

array(['Timberland'], dtype=object)